### Import

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

### Create variable

In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

In [3]:
train_data_dir = 'chest_xray/train'
validation_data_dir = 'chest_xray/val'
test_data_dir = 'chest_xray/test'

nb_train_samples = 5217
nb_validation_samples = 17
epochs = 20
batch_size = 16

In [6]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

### Create Sequential model

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

### Check information about model

In [9]:
model.inputs

[<tf.Tensor 'conv2d_1_input:0' shape=(?, 150, 150, 3) dtype=float32>]

In [10]:
model.output

<tf.Tensor 'activation_5/Sigmoid:0' shape=(?, 1) dtype=float32>

### Compile

In [11]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Upload img

In [12]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [13]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 5216 images belonging to 2 classes.


In [15]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 16 images belonging to 2 classes.


In [19]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 624 images belonging to 2 classes.


### Fit model

In [16]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/20
326/326 [==============================] - 302s 925ms/step - loss: 0.5079 - acc: 0.7889 - val_loss: 0.5957 - val_acc: 0.7500
Epoch 2/20
326/326 [==============================] - 274s 841ms/step - loss: 0.3560 - acc: 0.8633 - val_loss: 0.4023 - val_acc: 0.8125
Epoch 3/20
326/326 [==============================] - 259s 795ms/step - loss: 0.3224 - acc: 0.8779 - val_loss: 0.8220 - val_acc: 0.6250
Epoch 4/20
326/326 [==============================] - 259s 793ms/step - loss: 0.3256 - acc: 0.8861 - val_loss: 1.9381 - val_acc: 0.6250
Epoch 5/20
326/326 [==============================] - 259s 795ms/step - loss: 0.2987 - acc: 0.8982 - val_loss: 1.0933 - val_acc: 0.6875
Epoch 6/20
326/326 [==============================] - 256s 786ms/step - loss: 0.2668 - acc: 0.9005 - val_loss: 1.4881 - val_acc: 0.6250
Epoch 7/20
326/326 [==============================] - 264s 809ms/step - loss: 0.2823 - acc: 0.9036 - val_loss: 1.1046 - val_acc: 0.6250
Epoch 8/20
326/326 [============================

### Save model`s weights

In [17]:
model.save_weights('first_try.h5')

### Evalute model

In [21]:
# evaluate the model
scores = model.evaluate_generator(test_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


acc: 84.29%
